In [0]:
import pandas as pd
import folium
import json
import os
import pathlib as pl
from bs4 import BeautifulSoup
import urllib.request,json
import requests

In [232]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#scrape MOH website for real time info

url ='https://admin.jamcovid19.moh.gov.jm/public/api/statistics?type=1'

r = requests.get(url)
dictr = r.json()
recs = dictr['data']['parishes_wise_report']
df = pd.json_normalize(recs)
#print(df)

frame = pd.DataFrame(data=df)

#select columns needed
ParishData = frame.loc[:,['parish','total_tests_positive','number_of_deaths','number_of_people_recovered']]

#rename columns
ParishData.rename(columns={'parish':'Parish',
                           'total_tests_positive':'Confirmed','number_of_deaths':'Deaths', 
                           'number_of_people_recovered':'Recoveries'}, inplace=True)
ParishData


In [0]:
#use folium to get Jamaica's Map
jamaica = folium.Map(
    location=[18.1096,-77.2975],
    zoom_start=9,
    tiles='Mapbox Bright'
)

In [0]:
ParishData.dtypes

In [0]:
#change data types
ParishData['Parish'] = ParishData['Parish'].astype(str)

ParishData['Confirmed'] = pd.to_numeric(ParishData['Confirmed'])
ParishData['Recoveries'] = pd.to_numeric(ParishData['Recoveries'])
ParishData['Deaths'] = pd.to_numeric(ParishData['Deaths'])

ParishData.dtypes

In [0]:
#get geojosn file with Jamaica's Parishes/Boundaries
geojson = os.path.join(pl.Path.cwd(),'data', '/content/drive/My Drive/Colab Notebooks/Data/JMParishes.json')
jamaica_geo = json.load(open(geojson))

In [0]:
#create Choropleth Map of Jamaica 
jamaica.choropleth(
    geo_data = jamaica_geo,
    data = ParishData,
    columns = ['Parish','Confirmed'],
    key_on='feature.id',
    fill_color='YlOrRd',
    legend_name = 'Confirmed Corona Virus Cases by Parish'
)

# n = 0
# feature_group = folium.FeatureGroup("Locations")
# for i in jamaica_geo['features']:
#   feature_group.add_child(folium.Marker(location=[jamaica_geo['features'][n]['geometry']['coordinates'][0][0]], popup='Hello',tooltip='Parish',icon=folium.Icon(icon='cloud')))
#   #print(jamaica_geo['features'][n]['geometry']['coordinates'][0][0])
#   jamaica.add_child(feature_group)
#   n = n + 1
# print ('done')

folium.Marker(location=[18.0499998 -77.5333312],popup='Hello',tooltip='Parish').add_to(jamaica)
  
  


In [0]:
jamaica.save('jamaica.html')
jamaica